# Implementation of Intriguing Properties of NN (TensorFlow Implementation)

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


# Import initial dependancies
import numpy as np
import tensorflow as tf

tf.enable_eager_execution()

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

### Global vars

In [12]:
weight_decay = 0
num_epochs = 10
batch_size = 64

### Data Prep 

In [13]:
# mnist_train_ds, mnist_test_ds = datasets['train'], datasets['test']
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Use tf.data to batch and shuffle the ds
mnist_train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(1024).batch(32)

mnist_test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

### Define a FC model

In [14]:
from model import FC
fc_model = FC()

### Define a Loss and Optimizer

In [15]:
# Since there are 10 different classes in this ds, define loss as categorical
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Set the metrics to measure the loss and accuracy of the model
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


### Model training

In [16]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = fc_model(images)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, fc_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, fc_model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [17]:
@tf.function
def test_step(images, labels):
    predictions = fc_model(images)
    t_loss = loss_func(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [18]:
# Train the model
for epoch in range(num_epochs):
    # Reset the metrics at the start of each epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    
    
    for images, labels in mnist_train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in mnist_test_ds:
        test_step(test_images, test_labels)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    

Epoch 1, Loss: 0.2631697654724121, Accuracy: 92.54166412353516, Test Loss: 0.14277759194374084, Test Accuracy: 95.74000549316406
Epoch 2, Loss: 0.11875836551189423, Accuracy: 96.47000122070312, Test Loss: 0.10297231376171112, Test Accuracy: 96.8499984741211
Epoch 3, Loss: 0.07897233217954636, Accuracy: 97.66166687011719, Test Loss: 0.09144988656044006, Test Accuracy: 97.22000122070312
Epoch 4, Loss: 0.056857630610466, Accuracy: 98.3133316040039, Test Loss: 0.09022970497608185, Test Accuracy: 97.22999572753906
Epoch 5, Loss: 0.04220513626933098, Accuracy: 98.81832885742188, Test Loss: 0.08376258611679077, Test Accuracy: 97.44999694824219
Epoch 6, Loss: 0.03179953992366791, Accuracy: 99.1433334350586, Test Loss: 0.08287952840328217, Test Accuracy: 97.63999938964844
Epoch 7, Loss: 0.023859137669205666, Accuracy: 99.41166687011719, Test Loss: 0.08445628732442856, Test Accuracy: 97.68000030517578
Epoch 8, Loss: 0.0186361875385046, Accuracy: 99.5433349609375, Test Loss: 0.08940230309963226, 

In [19]:
fc_model.summary()

Model: "fc_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    multiple                  101770    
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
